In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import sys
sys.path.append("../input/tez-lib/")
sys.path.append("../input/timmmaster/")

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import torch
import torchvision
import sklearn
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision.io import read_image
import math

test_csv = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')
submission = pd.read_csv('../input/petfinder-pawpularity-score/sample_submission.csv')

test_dir = '../input/petfinder-pawpularity-score/test/'

def create_path(df,root_dir):
    df['Path'] = df['Id'].apply(lambda x: root_dir+x+'.jpg')
    
    
    
    
create_path(test_csv,test_dir)

# Create Label column (used later)
test_csv['Pawpularity'] = 100  

In [3]:
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

class CustomDataSet(Dataset):
    
    def __init__(self,csv_file,transform=None,augment_transform=None,root_dir ='../input/petfinder-pawpularity-score/train/' ):
        self.csv_file = csv_file
        self.transform = transform
        self.augment_trans = augment_transform
        self.root_dir = root_dir
        self.img_paths = self._get_img_paths(self.csv_file, root_dir)
        
    def __len__(self):
        return int(self.csv_file.shape[0])
    
    def __getitem__(self,idx):
        img_path = self.img_paths[idx]
        image = read_image(img_path)
        label = self.csv_file.Pawpularity.iloc[idx]/100
        image = image.type(torch.FloatTensor)
        
        if self.transform:
            image = self.transform(image)
        if self.augment_trans:
            image = self.augment_transform(image)
           
        image = torch.mul(image, (1/255))
        return {'image':image,'targets':label}
    
    def _get_img_paths(self,csv_file,root_dir):
        imgs = csv_file['Id'].apply(lambda x: root_dir+x+'.jpg').tolist()
        return imgs
    
    
 

from torch.utils.data.sampler import SubsetRandomSampler
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
transform = transforms.Compose([
                                normalize,
                                transforms.Resize(255),
                                transforms.CenterCrop(224)])



test_data = CustomDataSet(test_csv,transform,root_dir = test_dir)   

In [4]:
#!pip install timm
#!pip install tez
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)


import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import models
import time
import copy

import timm
from tez.callbacks import EarlyStopping


import tez
from tqdm import tqdm


cuda:0


In [5]:
import sys
sys.path.append("../input/tez-lib/")
sys.path.append("../input/timmmaster/")
import tez
import timm

from tez.callbacks import EarlyStopping
#sys.path.append("../input/tez-lib")
#sys.path.append("../input/timmmaster")
from sklearn import metrics

def mixup_loss(loss_fn, pred, y_a, y_b, lam):
    # get loss from current x n loss from watermarks n add
    return lam * loss_fn(pred, y_a) + (1 - lam) * loss_fn(pred, y_b)

class PawpularModel(tez.Model):
    def __init__(self,name):
        super().__init__()
        
        self.model = timm.create_model(name, pretrained=False, in_chans=3)
        self.model.head = nn.Linear(self.model.head.in_features, 128)
        self.dropout = nn.Dropout(p=0.5)
        self.dense1 = nn.Linear(128,1)
        
        self.step_scheduler_after = 'epoch'
        
    
    
    def forward(self, image, targets=None):        
        # do mixup when have targets and state is train (doesnt do mixup at val)
        if ((targets is not None) and (self._train_state == True)):
            #image, features, target_a, target_b, lam = mixup_data(image, features, targets.view(-1,1)) 
            image = image.to(device='cuda', dtype=torch.float)
            features = features.to(device='cuda', dtype=torch.float)
            target_a = target_a.to(device='cuda', dtype=torch.float)
            target_b = target_b.to(device='cuda', dtype=torch.float)
        x = self.model(image)
        x = self.dropout(x)
        # combine with meta features and shrink it down to 1 feature (score)
        
        x = self.dense1(x)
        
        
        
        return x, 0, {}

In [6]:
#model = timm.create_model("swin_large_patch4_window12_384",pretrained=True)

In [7]:
super_final_predictions = []
sys.path.append("../input/tez-lib/")
sys.path.append("../input/timmmaster/")
for fold_ in range(5):
    model = PawpularModel("swin_tiny_patch4_window7_224")
    model.load(f"../input/kfold-pet1/swin_tiny_patch4_window7_224_f{fold_}.bin", device="cuda", weights_only=True)

    df_test = pd.read_csv("../input/petfinder-pawpularity-score/test.csv")
    #test_img_paths = [f"../input/petfinder-pawpularity-score/test/{x}.jpg" for x in df_test["Id"].values]

    

    test_dataset = CustomDataSet(
        test_csv,transform,root_dir=test_dir
    )
    test_predictions = model.predict(test_dataset, batch_size=32, n_jobs=-1)

    final_test_predictions = []
    for preds in tqdm(test_predictions):
        final_test_predictions.extend(preds.ravel().tolist())

    final_test_predictions = [sigmoid(x) * 100 for x in final_test_predictions]
    super_final_predictions.append(final_test_predictions)

super_final_predictions = np.mean(np.column_stack(super_final_predictions), axis=1)
df_test["Pawpularity"] = super_final_predictions
df_test = df_test[["Id", "Pawpularity"]]
df_test.to_csv("submission.csv", index=False)

0it [00:00, ?it/s]
1it [00:06,  6.19s/it]
100%|██████████| 1/1 [00:06<00:00,  6.22s/it, stage=test]
1it [00:06,  6.23s/it]
0it [00:00, ?it/s]
1it [00:00,  7.26it/s]
100%|██████████| 1/1 [00:00<00:00,  5.43it/s, stage=test]
1it [00:00,  5.17it/s]
0it [00:00, ?it/s]
1it [00:00,  7.48it/s]
100%|██████████| 1/1 [00:00<00:00,  5.72it/s, stage=test]
1it [00:00,  5.41it/s]
0it [00:00, ?it/s]
1it [00:00,  7.30it/s]
100%|██████████| 1/1 [00:00<00:00,  5.51it/s, stage=test]
1it [00:00,  5.21it/s]
0it [00:00, ?it/s]
1it [00:00,  7.45it/s]
100%|██████████| 1/1 [00:00<00:00,  5.51it/s, stage=test]
1it [00:00,  5.20it/s]


In [8]:
df_test

,Id,Pawpularity
0,4128bae22183829d2b5fea10effdb0c3,43.230660
1,43a2262d7738e3d420d453815151079e,43.382045
2,4e429cead1848a298432a0acad014c9d,43.266073
3,80bc3ccafcc51b66303c2c263aa38486,43.188794
4,8f49844c382931444e68dffbe20228f4,43.222089
5,b03f7041962238a7c9d6537e22f9b017,43.341514
6,c978013571258ed6d4637f6e8cc9d6a3,43.213194
7,e0de453c1bffc20c22b072b34b54e50f,43.397801
